# XML example and exercise
****
+ study examples of accessing nodes in XML tree structure  
+ work on exercise to be completed and submitted
****
+ reference: https://docs.python.org/2.7/library/xml.etree.elementtree.html
+ data source: http://www.dbis.informatik.uni-goettingen.de/Mondial
****

In [1]:
from xml.etree import ElementTree as ET

## XML example

+ for details about tree traversal and iterators, see https://docs.python.org/2.7/library/xml.etree.elementtree.html

In [2]:
document_tree = ET.parse( './data/mondial_database_less.xml' )

In [3]:
# print names of all countries
for child in document_tree.getroot():
    print child.find('name').text

Albania
Greece
Macedonia
Serbia
Montenegro
Kosovo
Andorra


In [4]:
# print names of all countries and their cities
for element in document_tree.iterfind('country'):
    print '* ' + element.find('name').text + ':',
    capitals_string = ''
    for subelement in element.getiterator('city'):
        capitals_string += subelement.find('name').text + ', '
    print capitals_string[:-2]

* Albania: Tirana, Shkodër, Durrës, Vlorë, Elbasan, Korçë
* Greece: Komotini, Kavala, Athina, Peiraias, Peristeri, Acharnes, Patra, Kozani, Kerkyra, Ioannina, Thessaloniki, Iraklio, Chania, Ermoupoli, Rhodes, Tripoli, Lamia, Chalkida, Larissa, Volos, Mytilini, Karyes
* Macedonia: Skopje, Kumanovo
* Serbia: Beograd, Novi Sad, Niš
* Montenegro: Podgorica
* Kosovo: Prishtine
* Andorra: Andorra la Vella


****
## XML exercise

Using data in 'data/mondial_database.xml', the examples above, and refering to https://docs.python.org/2.7/library/xml.etree.elementtree.html, find

1. 10 countries with the lowest infant mortality rates
2. 10 cities with the largest population
3. 10 ethnic groups with the largest overall populations (sum of best/latest estimates over all countries)
4. name and country of a) longest river, b) largest lake and c) airport at highest elevation

In [5]:
document = ET.parse( './data/mondial_database.xml' )

## Exercise 1.  10 countries with the lowest infant mortality rates

In [10]:
import pandas as pd


In [13]:
df=pd.DataFrame(columns=['country','infant_mortality'])
for element in document.iterfind('country'):
    if element.find('infant_mortality') is not None:
        df2=pd.DataFrame([[element.find('name').text,element.find('infant_mortality').text]],columns=['country','infant_mortality'])
        df=df.append(df2)
df['infant_mortality']=pd.to_numeric(df['infant_mortality'],errors='coerce')
df.sort_values(by='infant_mortality').head(10)

,country,infant_mortality
0,Monaco,1.81
0,Japan,2.13
0,Bermuda,2.48
0,Norway,2.48
0,Singapore,2.53
0,Sweden,2.60
0,Czech Republic,2.63
0,Hong Kong,2.73
0,Macao,3.13
0,Iceland,3.15


## Exercise 2. 10  cities with the largest population

In [19]:
df_city=pd.DataFrame(columns=['city','population'])

for element in document.iterfind('country'):
    for subelement in element.getiterator('city'):
        if subelement.find('population') is not None:
            df_city2=pd.DataFrame([[subelement.find('name').text,subelement.find('population').text]],columns=['city','population'])
            df_city=df_city.append(df_city2)
df_city['population']=pd.to_numeric(df_city['population'],errors='coerce')
df_city.sort_values(by='population',ascending=False).head(10)

,city,population
0,Seoul,10229262
0,Mumbai,9925891
0,São Paulo,9412894
0,Jakarta,8259266
0,Shanghai,8205598
0,Ciudad de México,8092449
0,Moskva,8010954
0,Tokyo,7843000
0,Beijing,7362426
0,Delhi,7206704


## Exercise 3. 10 ethnic groups with the largest population sum over all country

In [23]:
df_ethnic=pd.DataFrame(columns=['country,ethnicgroup','population','percentage'])
for element in document.iterfind('country'): 
    for subelement in element.getiterator('ethnicgroup'):
        country=element.find('name').text
        ethnicgroup=subelement.text
        population=element.find('population').text
        percentage=subelement.attrib['percentage']
        df_ethnic2=pd.DataFrame([[country,ethnicgroup,population,percentage]],columns=['country','ethnicgroup','population','percentage'])
        df_ethnic=df_ethnic.append(df_ethnic2)

df_ethnic['population']=pd.to_numeric(df_ethnic['population'],errors='coerce')
df_ethnic['percentage']=pd.to_numeric(df_ethnic['percentage'],errors='coerce')
df_ethnic['population_per_ethnicity']=df_ethnic['population']*df_ethnic['percentage']/100
df_ethnic.groupby(['ethnicgroup']).sum().sort_values(by='population_per_ethnicity',ascending=False).head(10)

,percentage,population,population_per_ethnicity
ethnicgroup,,,
Han Chinese,91.50,543776080,4.975551e+08
European,970.82,362717873,1.928658e+08
Indo-Aryan,72.00,238396327,1.716454e+08
Russian,224.10,202263854,9.275844e+07
African,1868.55,357529690,8.632937e+07
Japanese,99.40,82199470,8.170627e+07
German,165.60,145710759,6.623219e+07
Dravidian,25.00,238396327,5.959908e+07
English,83.60,50616012,4.231499e+07


### Exercices 4: name and country of a) longest river, b) largest lake and c) airport at highest elevation

In [27]:
#a name and country of longest river
country_l_river=None
name_l_river=None
length_l_river=None
for node in document.iterfind('river'):
    if node.find('length')is not None:
        if length_l_river < float(node.find('length').text):
            length_l_river=float(node.find('length').text)
            country_l_river=node.attrib['country'] 
            name_l_river=node.attrib['id']
            
print('Longest river: ' +name_l_river+' in '+country_l_river)
              

Longest river:river-Amazonas in CO BR PE


In [29]:
#b name and country of b) largest lake
country_l_lake=None
name_l_lake=None
area_l_lake=None
for node in document.iterfind('lake'):
    if node.find('area') is not None:
        if area_l_lake<float(node.find('area').text):
            area_l_lake=float(node.find('area').text)
            country_l_lake=node.attrib['country']
            name_l_lake=node.attrib['id']
print('Largest lake: '+name_l_lake+ 'in' +country_l_lake )

Largest lake: lake-KaspischesMeerinR AZ KAZ IR TM


In [ ]:
#c airport at Highest elevation

In [35]:
elevation_h_airport=None
name_h_airport=None
country_h_airport=None
for node in document.iterfind('airport'):
    if node.find('elevation').text is not None:
        if elevation_h_airport < float(node.find('elevation').text):
            elevation_h_airport=float(node.find('elevation').text)
            country_h_airport=node.attrib['country']
            name_h_airport=node.find('name').text
print('Highest airport: ' +name_h_airport+'in'+ country_h_airport)


Highest airport: El Alto IntlinBOL
